In [1]:
import cv2
import numpy as np


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_dir = 'data/train'
val_dir = 'data/test'

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [7]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

In [8]:
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/20
448/448 [==============================] - 791s 2s/step - loss: 1.8339 - accuracy: 0.2362 - val_loss: 1.7424 - val_accuracy: 0.3177
Epoch 2/20
448/448 [==============================] - 223s 497ms/step - loss: 1.6726 - accuracy: 0.3449 - val_loss: 1.5553 - val_accuracy: 0.4100
Epoch 3/20
448/448 [==============================] - 229s 510ms/step - loss: 1.5516 - accuracy: 0.4054 - val_loss: 1.4800 - val_accuracy: 0.4400
Epoch 4/20
448/448 [==============================] - 227s 507ms/step - loss: 1.4911 - accuracy: 0.4271 - val_loss: 1.4337 - val_accuracy: 0.4556
Epoch 5/20
448/448 [==============================] - 243s 543ms/step - loss: 1.4321 - accuracy: 0.4527 - val_loss: 1.3833 - val_accuracy: 0.4766
Epoch 6/20
448/448 [==============================] - 258s 575ms/step - loss: 1.3785 - accuracy: 0.4733 - val_loss: 1.3382 - val_accuracy: 0.4980
Epoch 7/20
448/448 [==============================] - 234s 523ms/step - loss: 1.3260 - accuracy: 0.5018 - val_loss: 1.2970 - va

In [9]:
emotion_model.save_weights('emojify.h5')


In [17]:
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier(r'C:\Users\Sarvesh\haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
    break